# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [9]:
gmaps.configure(api_key=g_key)

In [10]:
weather_data = pd.read_csv("../starter_code/city_data_pd.csv")
weather_data = weather_data[['City','Country','Lat','Lng','Max Temp', 'Humidity','Wind Speed','Cloudiness','Date']]
weather_data

,City,Country,Lat,Lng,Max Temp,Humidity,Wind Speed,Cloudiness,Date
0,mar del plata,AR,-38.00,-57.56,55.99,28,17.22,0,1597681571
1,atuona,PF,-9.80,-139.03,78.24,73,17.16,0,1597681618
2,kamina,CD,-8.74,24.99,79.09,17,5.73,0,1597681816
3,vaini,TO,-21.20,-175.20,69.80,77,9.17,75,1597681816
4,bay roberts,CA,47.60,-53.26,78.01,41,6.22,80,1597681816
...,...,...,...,...,...,...,...,...,...
560,ferkessedougou,CI,9.60,-5.20,84.02,70,3.29,56,1597681933
561,kedrovyy,RU,56.17,91.82,57.20,76,2.24,0,1597681933
562,timra,SE,62.49,17.33,66.99,63,12.75,0,1597681933
563,cayenne,GF,4.93,-52.33,89.60,59,10.29,20,1597681933


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
locations = weather_data[["Lat", "Lng"]]
humidity = weather_data["Humidity"]
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
narrowcity_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 75) & (weather_data["Max Temp"] <= 80)].dropna()
narrowcity_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Wind Speed,Cloudiness,Date
2,kamina,CD,-8.74,24.99,79.09,17,5.73,0,1597681816
236,san quintin,MX,30.48,-115.95,77.50,64,6.20,0,1597681865
237,shache,CN,38.42,77.24,79.48,17,8.95,0,1597681865
314,hami,CN,42.80,93.45,79.29,16,5.48,0,1597681881
392,balkhash,KZ,46.84,74.98,75.78,28,6.82,0,1597681898


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
hotel_df = narrowcity_df
hotel_df["Hotel Name"] = ""

In [21]:
hotel_df

,City,Country,Lat,Lng,Max Temp,Humidity,Wind Speed,Cloudiness,Date,Hotel Name
2,kamina,CD,-8.74,24.99,79.09,17,5.73,0,1597681816,
236,san quintin,MX,30.48,-115.95,77.50,64,6.20,0,1597681865,
237,shache,CN,38.42,77.24,79.48,17,8.95,0,1597681865,
314,hami,CN,42.80,93.45,79.29,16,5.48,0,1597681881,
392,balkhash,KZ,46.84,74.98,75.78,28,6.82,0,1597681898,
417,zhoucheng,CN,35.91,116.31,79.65,92,5.75,0,1597681903,
452,foam lake,CA,51.65,-103.53,77.04,52,8.86,0,1597681911,


In [23]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
params ={
    "radius": 5000,
    "types": "lodging",
    "key": g_key}

for index, row in hotel_df.iterrows():
    lat = row.Lat
    lng = row.Lng
    params["location"] = f"{lat},{lng}"
    response = requests.get(url,params).json()
    try: 
        hotel_df.loc[index,"Hotel Name"] = response["results"][0]["name"]
    except(KeyError, IndexError):
        print("...missing field/results... skipping")
    

In [24]:
hotel_df

,City,Country,Lat,Lng,Max Temp,Humidity,Wind Speed,Cloudiness,Date,Hotel Name
2,kamina,CD,-8.74,24.99,79.09,17,5.73,0,1597681816,Guest House la Grace
236,san quintin,MX,30.48,-115.95,77.50,64,6.20,0,1597681865,Old Mill Hotel
237,shache,CN,38.42,77.24,79.48,17,8.95,0,1597681865,Delong Hotel
314,hami,CN,42.80,93.45,79.29,16,5.48,0,1597681881,Laoyutang Resort
392,balkhash,KZ,46.84,74.98,75.78,28,6.82,0,1597681898,Family
417,zhoucheng,CN,35.91,116.31,79.65,92,5.75,0,1597681903,Yingbin Restaurant
452,foam lake,CA,51.65,-103.53,77.04,52,8.86,0,1597681911,Foam Lake Campground


In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [28]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations,info_box_content = hotel_info)
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))